<a href="https://colab.research.google.com/github/baeksumin/k-project/blob/main/data_collection/datacollection_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
import re
import json
import pytz
import requests
import pandas as pd
from io import BytesIO
from datetime import datetime 
from datetime import timedelta
import pickle
from google.colab import drive
import matplotlib.pyplot as plt
import numpy as np
# import datetime

drive.mount('/content/drive')
data_path = '/content/drive/MyDrive/[Shared]K-Project/data/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
## Load pickle - fullcode 목록
with open(data_path + 'full_code.pkl',"rb") as f: 
    fullcode = pickle.load(f)
    
# 상장폐지 상태 제외시키기
fullcode_filter = fullcode[fullcode['status'] != '상장폐지']

# fullcode열을 list로 변경
fullcode_list = list(fullcode_filter['full_code'])
print(fullcode_list[:10])
print(len(fullcode_list))

['KR7060310000', 'KR7095570008', 'KR7006840003', 'KR7054620000', 'KR7265520007', 'KR7211270004', 'KR7027410000', 'KR7282330000', 'KR7138930003', 'KR7001460005']
2600


In [29]:
# 한국날짜 설정
def kor_date():
    tz = pytz.timezone('Asia/Seoul')  
    raw_dates = datetime.now(tz)
    yes = raw_dates - timedelta(1) 
    yesterday = yes.strftime("%Y%m%d")  
    return yesterday

# 조건에 해당하는 날짜 추출.
def get_point(result):
    x = list(result['x1'])+ list(result['x2'])
    x = list(set(x))
    x.sort()
    return x

# 전체 df에서 해당 날짜만 가져오기
def get_date(date_list):
    check_df = scode1[scode1.x == date_list[0]]

    for i in date_list :
        df = scode1[scode1.x == i]
        check_df = check_df.append(df,ignore_index = True)

    check_df = check_df.iloc[1:,:]
    return check_df


# 두 점 사이 관계 df 
def two_point (check_df):

    df = check_df[['x']]
    df= df.iloc[:-1]

    df[['x2','y1','y2','t','p','m']] = np.nan
    df.rename(columns ={'x':'x1'}, inplace = True)

    for i in range(len(df)): 
        df.iloc[i,1] = check_df.iloc[i+1,0]
        df.iloc[i,2] = check_df.iloc[i,1]
        df.iloc[i,3] = check_df.iloc[i+1,1]

    for i in range(df.shape[0]):

        t = df.iloc[i,1] - df.iloc[i,0]
        df.iloc[i,4] = t.days

        y1 = df.iloc[i,2]
        y2 = df.iloc[i,3]
        result = (abs(y2 - y1 )) / ((y1+y2)/2 )
        df.iloc[i,5] = result

        m = df.iloc[i,3]- df.iloc[i,2]    
        if (m>0):
            df.iloc[i,6] = 1
        elif (m<0):
            df.iloc[i,6] = -1
        else :
            df.iloc[i,6] = 0

    return df


# 조건 필터링 
def p_t(df):
    t = df['t'] <5
    p = df['p']<0.05

    result = df[~t&~p]
    return result


In [4]:
# 데이터 가져와서 필요 컬럼 추출 후 데이터프레임 생성하여 json으로 저장하는 함수
def make_initialdata(code):
    global result
    
    data2 = { 'bld': 'dbms/MDC/STAT/issue/MDCSTAT23902',
    'isuCd': code, 
    'isuCd2': '',
    'strtDd': strtDd,
    'endDd': endDd,
    'share': '1',
    'money': '1',
    'csvxls_isNo': 'false',}

    r2 = requests.post(url2, data2, headers)
    j2 = json.loads(r2.text)
    df2 = pd.DataFrame(j2['output'])
    # print(df2)

    data3 = { 'bld': 'dbms/MDC/STAT/standard/MDCSTAT03502',
    'isuCd': code,
    'isuCd2': code,
    'strtDd': strtDd,
    'endDd': endDd,
    'searchType': '2',
    'mktId': 'ALL',
    'csvxls_isNo': 'false',}

    r2 = requests.post(url2, data3, headers)
    j2 = json.loads(r2.text)
    df3 = pd.DataFrame(j2['output'])
    # print(df3)

    if len(df2.columns) > 0: # df2에 column이 존재하면 (df가 비어서 에러나는 경우 방지)
        df2_ = df2[['TRD_DD', 'ISU_CD', 'ISU_NM', 'TDD_CLSPRC', 'TDD_OPNPRC', 'TDD_HGPRC', 'TDD_LWPRC', 'MKTCAP']]
    else: # df가 비었을때 column 생성해줌
        df2_ = pd.DataFrame(index = range(0,len(df3)),
                           columns = ['TRD_DD', 'ISU_CD', 'ISU_NM', 'TDD_CLSPRC', 'TDD_OPNPRC', 'TDD_HGPRC', 'TDD_LWPRC', 'MKTCAP'])

    if len(df3.columns) > 0:
        df3_ = df3[['EPS', 'PER', 'BPS', 'PBR', 'DPS', 'DVD_YLD']]
    else:
        df3_ = pd.DataFrame(index = range(0,len(df2)),
                           columns = ['EPS', 'PER', 'BPS', 'PBR', 'DPS', 'DVD_YLD'])
 
    result = pd.concat([df2_, df3_], axis = 1)
    # print(result)

In [35]:
def make_trend(scode):
    global scode1

    #시간 순 재정렬.
    scode = scode.sort_values(by=['TRD_DD'])
    scode.reset_index(drop=True,inplace=True)
    scode['TRD_DD']=pd.to_datetime(scode['TRD_DD']) #datetime변환

    #시가총액 str->float 데이터타입변환
    scode['MKTCAP'] = scode['MKTCAP'].str.replace(',','').astype('float')

    #날짜, 시가총액 열만 추출
    scode1 = scode[['TRD_DD','MKTCAP']]
    scode1 = scode1.rename(columns = {'TRD_DD':'x','MKTCAP':'y'})
    scode1.reset_index(drop=True,inplace=True)


    #기울기 변하는 지점 찾아주기
    ischange = list()

    for i in range(1,len(scode1)-2):
      m1 = scode1.iloc[i,1] - scode1.iloc[i-1,1]
      m2 = scode1.iloc[i+1,1] - scode1.iloc[i,1]

      if(m1*m2<=0):
        ischange.append(scode1.iloc[i,0])

    # 전체 df에서 기울기 변하는 지점들만 추출한 후 , (t=5, p=0.05) 에 해당하는 날짜  추출.

    check_df = get_date(ischange) #전체 df에서 기울기 변하는 날짜만 추출하기.
    df = two_point(check_df)
    result = p_t(df)   # (t=5, p=0.05) 에 해당하는 날짜  추출.



    # 조건에 해당하는 날짜들끼리 다시 (t=5, p=0.05) 에 해당하는 날짜  추출. 
    red_x = get_point(result)
    red = get_date(red_x)
    df1=two_point(red)
    result2=p_t(df1)


    #점들이 모두 이어지고, 기울기가 계속 변하는 모습 나올때까지 반복 작업.
    while True:
      
      count = 0
      
      red_x1 = get_point(result2) #조건에 만족하는 날짜 추출

      for i in range(result2.shape[0]-1):
        a = result2.iloc[i,1] == result2.iloc[i+1,0]
        b = result2.iloc[i,6]* result2.iloc[i+1,6]== -1

        if(a&b ) :
          count+=1
        
      if (count ==  result2.shape[0]-1 ):
          print('\n'+fullcode_list[i]+" 조건 성립 완료 \n")
          break
          
      for i in range(result2.shape[0]-1):

        #i번째 기울기 음수일때
        if (result2.iloc[i,6]== -1):  
          #i+1번째 기울기 양수일 때
          if (result2.iloc[i+1,6] == 1): 
            #점이 이어져 있지 않으면
            if (result2.iloc[i,1] != result2.iloc[i+1,0]):
              if(result2.iloc[i,3]> result2.iloc[i+1,2]):
                red_x1.remove(result2.iloc[i,1])
              else :
                red_x1.remove(result2.iloc[i+1,0]) 
                
          #i+1번째 기울기 음수일 때
          elif (result2.iloc [i+1,6]== -1):       
            red_x1.remove(result2.iloc[i,1])


        #i번째 기울기 양수일때
        else :   
          #i+1번째 기울기 양수일 때
          if (result2.iloc[i+1,6] == 1): 
            #점이 이어져 있지 않으면
            if (result2.iloc[i,1] != result2.iloc[i+1,0]):
              red_x1.remove(result2.iloc[i,1])
              red_x1.remove(result2.iloc[i+1,0]) 
            #점이 이어져 있으면
            else :
              red_x1.remove(result2.iloc[i,1])
          #i+1번째 기울기 음수일 때
          else :
            #점이 이어져 있지 않으면
            if (result2.iloc[i,1] != result2.iloc[i+1,0]):
              if (result2.iloc[i,3]>=result2.iloc[i+1,2]):
                red_x1.remove(result2.iloc[i+1,0])
              else:
                red_x1.remove(result2.iloc[i,1])
        
      final = get_date(red_x1)
      df1=two_point(final)
      result2=p_t(df1)



    # trend -1~1 사이 값으로 변환.

    final = get_date(red_x1) #최종 기울기 변하는 점 추출.

    for i in range(final.shape[0]-1): #기울기 변하는 곳 1, -1로 값 채워주기
      if(result2.iloc[i,6]== 1):
        final.iloc[i,1] = -1
      else :
        final.iloc[i,1] = 1

    # 마지막 끝 점 (-1,1)해당하는 값으로 채워주기
    n = final.shape[0]-2
    if(final.iloc[n,1]== -1):
      final.iloc[final.shape[0]-1,1] = 1
    else:
      final.iloc[final.shape[0]-1,1] = -1

    # -1~ 1 사이 점 채워주기.(linear interpolation)
    scode_trend = scode1[['x']]
    scode_trend['TREND'] = np.nan

    for i in range(len(final)):
      scode_trend.loc[scode_trend['x']== final.iloc[i,0],'TREND'] = final.iloc[i,1]

    scode_trend =  scode_trend.set_index('x')
    scode_trend = scode_trend[final.iloc[0,0]:final.iloc[len(final)-1,0]].interpolate(method = "time")


    # 마지막으로 원래 데이터에 TREND 열 만들어주어서 합치기.
    scode_trend.reset_index(inplace=True)
    scode_trend = scode_trend.rename(columns = {'x':'TRD_DD'})
    scode = pd.merge(scode, scode_trend, on='TRD_DD', how='left')

    scode = scode.sort_values(by=['TRD_DD'], ascending=False)
    scode.reset_index(drop=True,inplace=True)

    print(scode)

In [8]:
with open(data_path + 'initial_data1.json',"r") as json_file:
    json_data = json.load(json_file) # json file open

In [36]:
pd.set_option('display.max_columns', 100)

url2 = 'http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd'
headers = {'User-Agent': 'Chrome/78.0.3904.87 Safari/537.36',}

key_list = list(json_data.keys())
result_dict = {}

global scode1

# for i in range(len(fullcode_list)): 
for i in range(5): 

    if fullcode_list[i] in key_list: # 어제 있던 데이터가 오늘도 정상적으로 있는 경우
        df = pd.read_json(json_data[fullcode_list[i]], orient ='index') # 첫번째 키값으로 데이터프레임 변환
        json_df = df.transpose()
        
        try:
            date = json_df.iloc[0,0] # 수집된 데이터의 마지막 날짜 확인
            yesterday = re.sub('/','-',date)
            yesterday = datetime.strptime(yesterday, '%Y-%m-%d') + timedelta(days = 1)
            yesterday = yesterday.strftime("%Y%m%d")

            strtDd = yesterday # 수집된 데이터의 마지막 날짜
            endDd = kor_date() # 10월 8일 00:30 코드실행이라면 10월 7일까지의 데이터 가져와야 함

            make_initialdata(fullcode_list[i])
            
            result1 = pd.concat([result, json_df], axis = 0, ignore_index = True)
            
            make_trend(result1)

            result_js = result1.to_json(orient = 'columns') # 데이터프레임을 json 형태로 변환
            result_dict[fullcode_list[i]] = result_js # fullcode가 key값이 되고 json이 value값이 됨

            print(i, '정상')
            print("\n-----------------------------------------------------------------------------------------\n")
        except:
            strtDd = 19000101
            endDd = kor_date()
            make_initialdata(fullcode_list[i])
            make_trend(result)

            result_js = result.to_json(orient = 'columns') # 데이터프레임을 json 형태로 변환
            result_dict[fullcode_list[i]] = result_js # fullcode가 key값이 되고 json이 value값이 됨

            print(i, 'except문 작동', fullcode_list[i])
            print("\n-----------------------------------------------------------------------------------------\n")

    elif fullcode_list[i] not in key_list: # 신규상장 된 경우
        strtDd = 19000101
        endDd = kor_date()
        make_initialdata(fullcode_list[i])
        make_trend(result)

        result_js = result.to_json(orient = 'columns') # 데이터프레임을 json 형태로 변환
        result_dict[fullcode_list[i]] = result_js # fullcode가 key값이 되고 json이 value값이 됨

        print(i, '신규상장', fullcode_list[i])
        print("\n-----------------------------------------------------------------------------------------\n")
    # elif key_list[i] not in fullcode_list: # 상장폐지 된 경우
    #     pass
    #     print(i, '상장폐지', key_list[i])

# # 최종 결과물 저장   
# with open(data_path + 'initial_data1.json', 'w') as f:
#     json.dump(result_dict, f)


KR7035250000 조건 성립 완료 

         TRD_DD  ISU_CD  ISU_NM TDD_CLSPRC TDD_OPNPRC TDD_HGPRC TDD_LWPRC  \
0    2021-11-11  060310      3S      3,945      3,860     3,980     3,860   
1    2021-11-10  060310      3S      3,945      4,045     4,045     3,920   
2    2021-11-09  060310      3S      4,020      4,000     4,040     3,985   
3    2021-11-08  060310      3S      4,005      3,990     4,095     3,990   
4    2021-11-05  060310      3S      3,985      4,080     4,130     3,960   
...         ...     ...     ...        ...        ...       ...       ...   
4831 2002-04-29  060310  삼에스코리아      6,780      6,800     7,200     6,780   
4832 2002-04-26  060310  삼에스코리아      7,700      8,550     8,750     7,700   
4833 2002-04-25  060310  삼에스코리아      8,750      9,530     9,530     7,490   
4834 2002-04-24  060310  삼에스코리아      8,510      8,370     8,510     7,750   
4835 2002-04-23  060310  삼에스코리아      7,600      7,600     7,600     7,600   

            MKTCAP  EPS     PER  BPS   PBR DPS DVD

In [ ]:
# 에러 발생 상황
# 신규상장되어 json data에 데이터가 없는 상황 --> if-else문으로 처리
# json data에 fullcode가 있는데 데이터프레임이 비어 있는 상황 --> try-except문으로 처리

In [ ]:
# # 저장된 데이터 가져와서 정상적으로 저장되었는지 테스트
# with open(data_path + 'initial_data1.json',"r") as json_file:
#     json_data = json.load(json_file) # json file open

# df = pd.read_json(json_data['KR7060310000'], orient ='index') # 첫번째 키값으로 데이터프레임 변환
# scode = df.transpose()
# print(scode)

In [ ]:
# 데이터가 정상적으로 들어간 것 확인!